## Test orc file

In [7]:
import pandas as pd
txt_path = '/Users/ymdt/src/dreem/test/test_files/bitvector/mysample/temp/AGGATTAG.txt'
output_path = '/Users/ymdt/src/dreem/test/test_files/bitvector/mysample/AGGATTAG.orc'
def make_bv_orc_from_txt(txt_path, output_path):
    with open(txt_path, 'r') as f:
        line = f.readline().split('\t')
        ref_seq = line[2]
    df = pd.read_csv(txt_path, sep='\t', skiprows=2).set_index('Query_name')
    df.drop('N_Mutations', axis=1, inplace=True)
    df = df.join(df['Bit_vector'].str.split('', expand=True)).drop(['Bit_vector',0,201], axis=1)
    df.columns = [str(c)+str(i) for i,c in enumerate(ref_seq)]
    df.to_orc(output_path, index=False)     

df = pd.read_orc(output_path)
df

,C0,C1,G2,A3,A4,A5,G6,T7,C8,A9,...,C190,T191,C192,C193,G194,T195,G196,T197,C198,T199
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,0,0,0,0,0,0,0,0,0,0,...,G,0,0,0,0,0,0,0,0,0
1996,0,0,0,0,0,0,0,0,0,0,...,G,0,0,0,0,0,0,0,0,0
1997,0,0,0,0,0,0,0,0,0,0,...,G,0,0,0,0,0,0,0,0,0
1998,0,0,0,0,0,0,0,0,0,0,...,G,0,0,0,0,0,0,0,0,0


## Test csv file

In [3]:
import sys, os
sys.path.append('/Users/ymdt/src/dreem/test')
from config import *
from generate_test_files import *
import pandas as pd
import numpy as np

cols_types = {
    'sample':str,
    'construct': str, 
    'cluster': int,
    'sequence':str,
    'num_reads': int,
    'num_aligned': int,
    'num_of_mutations': object,
    'mut_bases': object,
    'info_bases': object,
    'cov_bases': object,
    'del_bases': object,
    'ins_bases': object,
    'mod_bases_A': object,
    'mod_bases_C': object,
    'mod_bases_G': object,
    'mod_bases_T': object,
    'mut_rates': object,
    'worst_cov_bases': int,
    'skip_short_reads': int,
    'skip_too_many_muts': int,
    'skip_low_mapq': int
    }

df = pd.DataFrame({k:pd.Series([], dtype=v) for k,v in cols_types.items()}, index=np.arange(10))

df['sample'] = SAMPLE_NAME
df['construct'] = CONSTRUCTS
df['sequence'] = [build_sequence_from_barcode(barcode, SEQUENCE, BARCODE_START, BARCODE_END) for barcode in BARCODES]
df['worst_cov_bases'] = NUM_READS
df['skip_short_reads'] = 0
df['skip_too_many_muts'] = 0
df['skip_low_mapq'] = 0

lib = pd.read_csv('/Users/ymdt/src/dreem/test/test_files/library.csv', index_col=0).reset_index().rename(columns={'section_name':'section'})
df = df.merge(lib, on='construct', how='left')
samples = pd.read_csv('/Users/ymdt/src/dreem/test/test_files/samples.csv', index_col=0).reset_index()
df = df.merge(samples, on='sample', how='left')

reads_array = np.zeros((NUM_READS*2,LENGTH_SEQUENCE))
for read in [0,1]:
    for r in range(NUM_READS):
        reads_array[r+read*NUM_READS] = add_mutation(r, np.zeros(LENGTH_SEQUENCE, dtype=int))
print(reads_array.sum(axis=1))

num_reads = {
    'pop_avg': NUM_READS,
    '1': NUM_READS//2,
    '2': NUM_READS//2 - NUM_READS//10,
    '3': NUM_READS//10,
}
mutation_points = MUTATION_THRESHOLD.values()

num_of_mutations = {
    'pop_avg':np.zeros(len(SEQUENCE), dtype=int),
    '1': np.zeros(len(SEQUENCE), dtype=int),
    '2': np.zeros(len(SEQUENCE), dtype=int),
    '3':np.zeros(len(SEQUENCE), dtype=int),
}
for k, v in num_of_mutations.items():
    num_of_mutations[k][1] = num_reads[k]

mut_bases = {
    'pop_avg':np.zeros(len(SEQUENCE), dtype=int),
    '1': np.zeros(len(SEQUENCE), dtype=int),
    '2': np.zeros(len(SEQUENCE), dtype=int),
    '3':np.zeros(len(SEQUENCE), dtype=int),
}


df_out = pd.DataFrame()
for cluster in ['pop_avg','1','2','3']:
    df_temp  = df.copy()
    df_temp['cluster'] = cluster
    df_temp['mut_bases'] = df_temp['mut_bases'].astype(object)
    df_temp['num_reads'], df_temp['num_aligned'] = num_reads[cluster], num_reads[cluster]
    df_temp['num_of_mutations'] = [num_of_mutations[cluster]]*len(df_temp)
    df_out = pd.concat([df_out, df_temp])
df = df_out.reset_index(drop=True)
df.to_csv('/Users/ymdt/src/dreem/test/test_files/'+SAMPLE_NAME+'.csv', index=False)

df



KeyError: 0